# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Análisis de grandes volúmenes de datos (Gpo 10)**

### Tecnológico de Monterrey

## Actividad 3

###  Aprendizaje supervisado y no supervisado


#### **Nombrey matrícula**

*   **A01746998** - Alexys Martín Coate Reyes


# **1. Introducción teórica**



# **2. Selección de los datos**

### Cargando los datos

In [1]:
# Librerias
from pyspark.sql import SparkSession

In [2]:
# Crear sesión Spark
spark = SparkSession.builder \
    .appName("EDA_Vuelos") \
    .getOrCreate()

# Leer el CSV
df = spark.read.csv("../Dataset/Airline_Delay_2016-2018.csv", header=True, inferSchema=True)

# Mostrar esquema de datos
df.printSchema()

# Número total de registros
total_registros = df.count()
print(f"Número total de registros: {total_registros}")

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)

Número total de registros: 18505725


In [3]:
# Imprimiendo los 3 primeros rengloes del dataframe dataframe
df.show(3)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+
|2016-01-01|        DL|             1248|   DTW| LAX|        1935|  1935.0|      0.0|    23.0|    1958.0|   2107.0|   13.0|        2144|  2120.0|    -24.0|      0.0|     0.0|           309.0|              285.0|   249.0|  1979.0|
|2016-01-01|        DL|             1251|   ATL| GRR|        2125|  2130.0|     

### Seleccionando variables de caracterización 

In [4]:
# Variables seleccionadas
vars_particion = ["OP_CARRIER", "ORIGIN", "DEST", "CANCELLED", "DIVERTED"]

# Imrpimiendo la cantidad de valores únicos que se tiene por las columnas seleccionadas
for col in vars_particion:
    print(col, df.select(col).distinct().count())

OP_CARRIER 18
ORIGIN 362
DEST 360
CANCELLED 2
DIVERTED 2


In [5]:
from pyspark.sql.functions import col, count, round

# Calcular frecuencia de combinaciones
combinaciones = df.groupBy(vars_particion).count()

# Total de registros
total = df.count()

# Agregar probabilidad
combinaciones = combinaciones.withColumn("probabilidad", col("count") / total)

combinaciones.show(truncate=False)

+----------+------+----+---------+--------+-----+---------------------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|count|probabilidad         |
+----------+------+----+---------+--------+-----+---------------------+
|EV        |ATL   |AGS |0.0      |0.0     |2912 |1.573567098830227E-4 |
|EV        |DTW   |BWI |0.0      |0.0     |23   |1.2428586288837644E-6|
|EV        |BRO   |IAH |0.0      |0.0     |3054 |1.6503001098308766E-4|
|F9        |DEN   |MIA |0.0      |0.0     |1089 |5.884665421106171E-5 |
|WN        |BDL   |RSW |0.0      |0.0     |735  |3.971743879258986E-5 |
|WN        |BWI   |MEM |0.0      |0.0     |1348 |7.284232311892671E-5 |
|WN        |DEN   |BOS |0.0      |0.0     |2467 |1.3331009728070637E-4|
|WN        |LAX   |LAS |0.0      |0.0     |10922|5.901957367247162E-4 |
|WN        |LAX   |SAT |0.0      |0.0     |2056 |1.1110075395587041E-4|
|WN        |MCO   |CMH |0.0      |0.0     |3558 |1.922648261551493E-4 |
|WN        |PIT   |DEN |0.0      |0.0     |1307 |7.0626792519612

In [6]:
from pyspark.sql.functions import when, lit

# Definir tamaño total de la muestra (por ejemplo, 1% del total)
tamaño_muestra_total = int(total * 0.01)

# Establecer un mínimo de registros por partición
minimo_por_particion = 0

# Calcular el tamaño de muestra por partición según su probabilidad
combinaciones_con_tamaño = combinaciones.withColumn(
    "tamaño_muestra",
    round(
        when(
            (col("probabilidad") * tamaño_muestra_total) < minimo_por_particion,
            lit(minimo_por_particion)
        ).otherwise(col("probabilidad") * tamaño_muestra_total)
    ).cast("int")
)

# Ordenar para visualizar mejor
combinaciones_con_tamaño = combinaciones_con_tamaño.orderBy(col("tamaño_muestra").desc())

combinaciones_con_tamaño.show(10, truncate=False)

# Calcular el tamaño final total de la muestra
tamaño_muestra_final = combinaciones_con_tamaño.agg({"tamaño_muestra": "sum"}).collect()[0][0]

print(f"Tamaño total esperado de la muestra final: {tamaño_muestra_final}")

+----------+------+----+---------+--------+-----+---------------------+--------------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|count|probabilidad         |tamaño_muestra|
+----------+------+----+---------+--------+-----+---------------------+--------------+
|HA        |OGG   |HNL |0.0      |0.0     |28891|0.0015611925498730799|289           |
|HA        |HNL   |OGG |0.0      |0.0     |28830|0.0015578962726399534|288           |
|HA        |KOA   |HNL |0.0      |0.0     |21350|0.0011536970315942769|213           |
|HA        |HNL   |KOA |0.0      |0.0     |20673|0.0011171137580397417|207           |
|HA        |HNL   |LIH |0.0      |0.0     |19946|0.0010778286179006767|199           |
|HA        |LIH   |HNL |0.0      |0.0     |19901|0.0010753969379745998|199           |
|WN        |DAL   |HOU |0.0      |0.0     |19109|0.0010325993712756458|191           |
|WN        |HOU   |DAL |0.0      |0.0     |19131|0.0010337881925728389|191           |
|DL        |MCO   |ATL |0.0      |0.0     |

In [7]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Unir el tamaño de muestra a cada combinación en el dataset original
df_con_muestra = df.join(
    combinaciones_con_tamaño.select(*vars_particion, "tamaño_muestra"),
    on=vars_particion,
    how="inner"
)

# Crear ventana ordenada por fecha y hora dentro de cada combinación
ventana = Window.partitionBy(*vars_particion).orderBy("FL_DATE", "CRS_DEP_TIME")

# Enumerar vuelos por combinación (ordenados por tiempo)
df_con_muestra = df_con_muestra.withColumn("row_num", row_number().over(ventana))

# Filtrar solo los primeros N registros por combinación
muestra_final = df_con_muestra.filter(col("row_num") <= col("tamaño_muestra"))

# Mostrar la muestra final
muestra_final.show(10, truncate=False)

print(f"Tamaño total de la muestra: {muestra_final.count()}")

+----------+------+----+---------+--------+----------+-----------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+----------------+-------------------+--------+--------+--------------+-------+
|OP_CARRIER|ORIGIN|DEST|CANCELLED|DIVERTED|FL_DATE   |OP_CARRIER_FL_NUM|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|tamaño_muestra|row_num|
+----------+------+----+---------+--------+----------+-----------------+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+----------------+-------------------+--------+--------+--------------+-------+
|9E        |ATL   |AGS |0.0      |0.0     |2018-01-01|3296             |855         |854.0   |-1.0     |12.0    |906.0     |939.0    |3.0    |953         |942.0   |-11.0    |58.0            |48.0               |33.0    |143.0   |10          

# **3. Preparación de los datos**

### Selección de variables importantes para el modelo

Se realizará un análisis de predicción para saber que tanto un avión se retrasará. Por ello se quitarán las columnas que no aportan mucha información para el modelo.

In [8]:
import pandas as pd

pandas_df = muestra_final.limit(50).toPandas()
display(pandas_df)

,OP_CARRIER,ORIGIN,DEST,CANCELLED,DIVERTED,FL_DATE,OP_CARRIER_FL_NUM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,tamaño_muestra,row_num
0,9E,ATL,AGS,0.0,0.0,2018-01-01,3296,855,854.0,-1.0,...,3.0,953,942.0,-11.0,58.0,48.0,33.0,143.0,10,1
1,9E,ATL,AGS,0.0,0.0,2018-03-01,3680,1355,1534.0,99.0,...,3.0,1445,1636.0,111.0,50.0,62.0,45.0,143.0,10,2
2,9E,ATL,AGS,0.0,0.0,2018-03-02,3366,1746,1752.0,6.0,...,3.0,1845,1843.0,-2.0,59.0,51.0,30.0,143.0,10,3
3,9E,ATL,AGS,0.0,0.0,2018-03-02,3809,2134,2132.0,-2.0,...,5.0,2232,2226.0,-6.0,58.0,54.0,30.0,143.0,10,4
4,9E,ATL,AGS,0.0,0.0,2018-03-03,3354,1026,1019.0,-7.0,...,4.0,1122,1115.0,-7.0,56.0,56.0,31.0,143.0,10,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,AA,LGA,MIA,0.0,0.0,2016-01-07,2266,640,632.0,-8.0,...,58.0,951,1027.0,36.0,191.0,235.0,158.0,1096.0,116,66
996,AA,LGA,MIA,0.0,0.0,2016-01-07,1410,815,813.0,-2.0,...,44.0,1138,1152.0,14.0,203.0,219.0,157.0,1096.0,116,67
997,AA,LGA,MIA,0.0,0.0,2016-01-07,1103,1000,1000.0,0.0,...,5.0,1321,1247.0,-34.0,201.0,167.0,147.0,1096.0,116,68
998,AA,LGA,MIA,0.0,0.0,2016-01-07,1082,1100,1103.0,3.0,...,7.0,1421,1356.0,-25.0,201.0,173.0,151.0,1096.0,116,69


In [9]:
# Se crea una nueva columna para verificar si 
delayed_time = 10
df_muestra = muestra_final.withColumn("DELAYED", when(df.ARR_DELAY > delayed_time, 1).otherwise(0))

In [10]:
# Variables a descartar
cols_to_drop = ["tamaño_muestra", "row_num", "OP_CARRIER_FL_NUM", "DEP_TIME","DEP_DELAY", "TAXI_OUT", 
                "WHEELS_OFF", "WHEELS_ON", "TAXI_IN" , "ARR_TIME", "ARR_DELAY", "DIVERTED", "ACTUAL_ELAPSED_TIME", "AIR_TIME"]

# Variables importantes para el modelo
selected_cols = [
    "FL_DATE", "OP_CARRIER", "ORIGIN", "DEST",
    "CRS_DEP_TIME", "CRS_ARR_TIME", "CRS_ELAPSED_TIME",
    "DISTANCE", "CANCELLED", "ARR_DELAY", "DELAYED"
]

# Realizando la selección de columnas importantes como datos de entrenamiento
df_raw = df_muestra.select(*selected_cols)
df_raw.show()

+----------+----------+------+----+------------+------------+----------------+--------+---------+---------+-------+
|   FL_DATE|OP_CARRIER|ORIGIN|DEST|CRS_DEP_TIME|CRS_ARR_TIME|CRS_ELAPSED_TIME|DISTANCE|CANCELLED|ARR_DELAY|DELAYED|
+----------+----------+------+----+------------+------------+----------------+--------+---------+---------+-------+
|2018-01-01|        9E|   ATL| AGS|         855|         953|            58.0|   143.0|      0.0|    -11.0|      0|
|2018-03-01|        9E|   ATL| AGS|        1355|        1445|            50.0|   143.0|      0.0|    111.0|      1|
|2018-03-02|        9E|   ATL| AGS|        1746|        1845|            59.0|   143.0|      0.0|     -2.0|      0|
|2018-03-02|        9E|   ATL| AGS|        2134|        2232|            58.0|   143.0|      0.0|     -6.0|      0|
|2018-03-03|        9E|   ATL| AGS|        1026|        1122|            56.0|   143.0|      0.0|     -7.0|      0|
|2018-03-03|        9E|   ATL| AGS|        1746|        1842|           

### Transformación de variables

In [11]:
from pyspark.sql.functions import dayofweek, month
from pyspark.ml.feature import StringIndexer

# Extrayendo el día, mes y eliminando la columna original de fecha
df_transformed = df_raw.withColumn("FL_DAY_OF_WEEK", dayofweek("FL_DATE")) \
                     .withColumn("FL_MONTH", month("FL_DATE")) \
                     .drop("FL_DATE") 



# Codificaando las variables categóricas
indexers = [
    StringIndexer(inputCol="OP_CARRIER", outputCol="OP_CARRIER_T"),
    StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_T"),
    StringIndexer(inputCol="DEST", outputCol="DEST_T")
]

for indexer in indexers:
    df_transformed = indexer.fit(df_transformed).transform(df_transformed)

# Eliminando las columnas de las variables categóricas originales
cols_to_drop = ["OP_CARRIER", "ORIGIN", "DEST"]
df_transformed = df_transformed.drop(*cols_to_drop) # Using * to unpack the list of column names

df_transformed.show()

ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión

In [ ]:
import os
os.environ["HADOOP_HOME"] = "C:/hadoop"
os.environ["hadoop.home.dir"] = "C:/hadoop"

In [ ]:
df_transformed.write.mode("overwrite").parquet("./df_transformed.parquet")

In [ ]:
df_transformed.printSchema()

In [ ]:
df_transformed.describe().toPandas()

### Limpieza de datos

In [ ]:
#Se eliminan registros con valores nulos
df_clean = df_transformed.dropna()

#Se eliminan columnas con valores nulos
df_clean = df_clean.na.drop()

#Se eliminan registros duplicados
df_clean = df_clean.dropDuplicates()

In [ ]:
df_clean.describe().toPandas()

# **4. Prepraración del conjunto de entrenamiento y prueba**

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[
        "OP_CARRIER_T","ORIGIN_T","DEST_T",
        "CRS_DEP_TIME","CRS_ARR_TIME","CRS_ELAPSED_TIME",
        "DISTANCE","CANCELLED",
        "FL_DAY_OF_WEEK", "FL_MONTH"
        #"ARR_DELAY","DELAYED"
    ],
    outputCol="features"
)

df_vector = assembler.transform(df_clean)

In [ ]:
# Separación de datos de entrenamiento y prueba
df_train, df_val_test = df_vector.randomSplit([0.7, 0.3], seed=42)
df_val, df_test = df_val_test.randomSplit([0.5, 0.5], seed=42)

# **5. Construcción de modelos de aprendizaje supervisado y no supervisado**

### Modelo Supervisado - Regresión Logística

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="DELAYED")
lr_model = lr.fit(train_df)

### Modelo No Supervisado - 